## Lab 3: Isomap
You can use external libraries for linear algebra operations but you are expected to write your own algorithms.

### Notes
- You can use the ```sklearn.neighbors.NearestNeighbors``` class.
- You are expected to write your own implementation of the Floyd-Warshall algorithm, as seen in class.
- Test your algorithm with $n=100$ points at first.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder

# Exercise 1
- Write your own implementation of Isomap. 
- Apply it to the Swiss Roll dataset ($n=1000$) from Lab 1. 

In [ ]:
def swiss_roll(n): #from lab 1
    """
    Parameters:
    n: int
        Number of points to generate"""
    
    data = np.zeros((n,3))
    phi = np.random.uniform(low=1.5*np.pi, high=4.5*np.pi, size=n)
    psi = np.random.uniform(0,10,n)
            
    data[:,0]=phi*np.cos(phi) #x coordinte
    data[:,1]=phi*np.sin(phi) #y coordinate
    data[:,2]=psi #z coordinate
    return data

In [ ]:
from sklearn.manifold import Isomap

In [ ]:
embedding = Isomap(n_components=2, path_method='FW') #n_neighbors=5 (default)

#how to choose k?
#rule of thumb: select it as the smallest k that guarantees a fully-connected graph
#if it is set too large, you will have shortcuts

In [ ]:
X = swiss_roll(1000)

In [ ]:
plt.figure(figsize=(12,10))
axes = plt.axes(projection='3d')
print(type(axes))
axes.scatter3D(X[:,0], X[:,1], X[:,2],c=X[:,0])

axes.set_xlabel('x')
axes.set_ylabel('y')
axes.set_zlabel('z')

#axes.view_init(45, 45) #(elevation, azimuth angle)

plt.show()

In [ ]:
X_new = embedding.fit_transform(X)

In [ ]:
X_new.shape

In [ ]:
plt.plot(X_new[:,0], X_new[:,1], 'o')

If we test it with a larger number of datapoints (say $n=5000$) we can see that the algorithm works much better. 
The Swiss Roll is denser and the Isomap projection is more clearly able to unfold the manifold. With the ```sklearn``` implementation it takes roughly 30 seconds, it may be that your version is slower. 

In [ ]:
embedding = Isomap(n_components=2, path_method='FW') #n_neighbors=5 (default)

In [ ]:
X = swiss_roll(5000)

In [ ]:
plt.figure(figsize=(12,10))
axes = plt.axes(projection='3d')
print(type(axes))
axes.scatter3D(X[:,0], X[:,1], X[:,2],c=X[:,0])

axes.set_xlabel('x')
axes.set_ylabel('y')
axes.set_zlabel('z')

#axes.view_init(45, 45) #(elevation, azimuth angle)

plt.show()

In [ ]:
X_new = embedding.fit_transform(X)

In [ ]:
plt.plot(X_new[:,0], X_new[:,1], 'o')

- Use a modified version of the Swiss Roll dataset, in which Gaussian noise from a Normal $N(\mu=0, \sigma=0.5)$ is added to the $x$ and $y$ coordinates. Apply Isomap to this dataset and discuss the differences with the previous point.

In [ ]:
def swiss_roll_noisy(n):
    """
    Parameters:
    n: int
        Number of points to generate"""
    
    data = np.zeros((n,3))
    phi = np.random.uniform(low=1.5*np.pi, high=4.5*np.pi, size=n)
    psi = np.random.uniform(0,10,n)
            
    data[:,0]=phi*np.cos(phi) + np.random.normal(0,0.5,n) #x coordinte
    data[:,1]=phi*np.sin(phi) + np.random.normal(0,0.5,n) #y coordinate
    data[:,2]=psi #z coordinate
    return data

In [ ]:
X = swiss_roll_noisy(1000)

In [ ]:
plt.plot(X[:,0], X[:,1], 'o')
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
axes = plt.axes(projection='3d')
print(type(axes))
axes.scatter3D(X[:,0], X[:,1], X[:,2], c=X[:,0])

axes.set_xlabel('x')
axes.set_ylabel('y')
axes.set_zlabel('z')

axes.view_init(45, 60) #(elevation, azimuth angle)

plt.show()

In [ ]:
embedding = Isomap(n_components=2,path_method='FW')

In [ ]:
X_new = embedding.fit_transform(X)

In [ ]:
plt.plot(X_new[:,0], X_new[:,1], 'o')

# Exercise 2
Undersample randomly from the  ```Dry_Bean_Dataset.xlsx``` in order to have $n=1000$ datapoints. Follow the same pipeline of Exercise 1 of the previous lab by replacing PCA with Isomap. Discuss the differences with particular focus on the accuracy of the logistic regression.

In [ ]:
df = pd.read_excel("../Datasets/Dry_Bean_Dataset.xlsx")

In [ ]:
y = df['Class']
X = df.drop('Class', axis=1)

In [ ]:
encoder = OrdinalEncoder()
y=np.array(y)
encoder.fit(y.reshape(-1,1))
y = encoder.transform(y.reshape(-1, 1))
print(y)  

In [ ]:
X.shape

In [ ]:
X, _, y , _ = train_test_split(X, y, train_size =1000)

In [ ]:
X.shape

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.8)

In [ ]:
X_mean = Xtrain.mean()
X_std = Xtrain.std()

In [ ]:
Xtrain = (Xtrain-X_mean )/X_std

In [ ]:
isomap = Isomap(n_components=16, path_method='FW')

In [ ]:
X_new = isomap.fit_transform(Xtrain)

In [ ]:
Xtest = (Xtest- Xtest.mean())/Xtest.std()

In [ ]:
score = []
for i in range(X.shape[1]):
    lr = LogisticRegression(multi_class='multinomial', max_iter=1000)
    lr.fit(X_new[:,:i+1], ytrain.ravel())

    x_PC = isomap.transform(Xtest)
    x_PC = x_PC[:,:i+1] #keep only
    
    yhat = lr.predict(x_PC)
    print(lr.score(x_PC, ytest.ravel()))
    score.append(lr.score(x_PC, ytest.ravel()))

In [ ]:
plt.plot(score)
plt.show()

In [ ]:
print(f"The maximum values of the accuracy score is reached with {np.argmax(score)} PCs and it is equal to {np.max(score)}")